In [1]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline


import jax.numpy as np
import pickle
import itertools
'unset XLA_FLAGS'

from chainconsumer import ChainConsumer
from chainconsumer.analysis import Analysis

Populating the interactive namespace from numpy and matplotlib


## Load data

In [3]:

with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_mse.pkl", 'rb') as g:
    sample_nd_mse = pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_mae.pkl", 'rb') as g:
    sample_nd_mae= pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_vmim.pkl", 'rb') as g:
    sample_nd_vmim = pickle.load(g)

sample_ff=np.load("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/posterior_full_field__256N_10ms_27gpa_0.26se.npy",allow_pickle=True)

sample_ps=np.load("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/posterior_power_spectrum__256N_10ms_27gpa_0.26se.npy",allow_pickle=True)


In [4]:
name_parameters= ['Omegac', 'Omegab', 'sigma8', 'h0', 'ns', 'w0']

In [5]:
sample=sample_nd_mae
c = ChainConsumer()
c.add_chain([sample[:,i] for i in range(6)], parameters= name_parameters)

In [6]:

lis=[0,1,2,3,4,5]

f_o_m= {}
for i, j in itertools.combinations(lis, 2):
    cov=c.analysis.get_covariance(parameters=[name_parameters[i],name_parameters[j]])
    fom=sqrt(np.linalg.det(np.linalg.pinv(cov[1])))
    name=cov[0]
    f_o_m[str(name)] = fom
            

In [7]:
f_o_m

{"['Omegac', 'Omegab']": 5908.9604,
 "['Omegac', 'sigma8']": 2315.777,
 "['Omegac', 'h0']": 617.9329,
 "['Omegac', 'ns']": 1030.1096,
 "['Omegac', 'w0']": 161.73602,
 "['Omegab', 'sigma8']": 5397.5273,
 "['Omegab', 'h0']": 2979.3105,
 "['Omegab', 'ns']": 4259.6743,
 "['Omegab', 'w0']": 771.26935,
 "['sigma8', 'h0']": 571.0067,
 "['sigma8', 'ns']": 854.49286,
 "['sigma8', 'w0']": 149.45586,
 "['h0', 'ns']": 469.14944,
 "['h0', 'w0']": 81.66431,
 "['ns', 'w0']": 119.41665}

In [8]:
np.sqrt(np.diag(c.analysis.get_covariance(parameters=name_parameters)[1]))

Array([0.02873233, 0.00596861, 0.03104132, 0.05652963, 0.03950926,
       0.21804836], dtype=float32)

In [9]:
name_parameters

['Omegac', 'Omegab', 'sigma8', 'h0', 'ns', 'w0']

In [10]:
c.analysis.get_summary()

{'Omegac': [0.2515119406304733, 0.27925633710102643, 0.3091056557900951],
 'Omegab': [0.0440492569504775, 0.050136666979554115, 0.05603591145217641],
 'sigma8': [0.7781923057101449, 0.808222208515031, 0.8404073675020861],
 'h0': [0.6063696445283618, 0.6639803448015603, 0.7199920937635598],
 'ns': [0.9135459113505426, 0.9546853835277671, 0.9933437465435291],
 'w0': [-1.378711914001921, -1.1594029863325186, -0.9414163347128024]}

In [11]:
c.analysis.get_latex_table( transpose=True)


'\\begin{table}\n    \\centering\n    \\caption{}\n    \\label{tab:model_params}\n    \\begin{tabular}{cc}\n        \\hline\n\t\tParameter & Chain 0 \\\\ \n\t\t\\hline\n\t\tOmegac & $0.279^{+0.030}_{-0.028}$ \\\\ \n\t\tOmegab & $\\left( 50.1^{+5.9}_{-6.1} \\right) \\times 10^{-3}$ \\\\ \n\t\tsigma8 & $0.808^{+0.032}_{-0.030}$ \\\\ \n\t\th0 & $0.664^{+0.056}_{-0.058}$ \\\\ \n\t\tns & $0.955^{+0.039}_{-0.041}$ \\\\ \n\t\tw0 & $-1.16\\pm 0.22$ \\\\ \n\t\t\\hline\n    \\end{tabular}\n\\end{table}'